In [ ]:
from datetime import datetime, timedelta
from pyspark import SparkContext, SQLContext
from pyspark.sql.functions import hour, mean
import pyspark
import random
sc = SparkContext.getOrCreate()

In [ ]:
spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('auctions.csv', header=True).limit(10000)
rdd_auctions = df_auctions.rdd
rdd_auctions.take(5)

In [ ]:
def didConvert():
    return bool(random.getrandbits(1))

In [ ]:
rdd_auctions = rdd_auctions.map(lambda x: (x[1], (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), didConvert())))

In [ ]:
rdd_train = rdd_auctions.groupByKey().mapValues(list).mapValues(sorted)

In [ ]:
rdd_train.take(20)

In [ ]:
def get_convertion_timeDeltas(dateList):
    """
    esta funcion recibe una lista de fechas ordenada de las apariciones de un ID
    la idea es devolver una lista de tuplas, en la que cada tupla sea asi:
    valor 1 = fecha en que aparecio en un auction
    valor 2 = distancia temporal hasta la siguiente fecha donde convirtió (0: si convirtió en esa fecha,
                                                                            3 días: si no convirtió 
                                                                            en una fecha posterior)
    """
    distancias = []
    longitud_actual = len(dateList)
    for x in range(longitud_actual):
        for y in range(x, longitud_actual):
            if(y == longitud_actual - 1 and dateList[y][1] != True):
                distancias.append([dateList[x][0], dateList[x][1],
                                            timedelta(days=3).total_seconds()
                                        ])
                break
            elif(dateList[y][1] == True):
                distancias.append([dateList[x][0], dateList[x][1],
                                    (dateList[y][0] - dateList[x][0]).total_seconds()
                                    ])
                break
        
    return distancias

In [ ]:
rdd_train_test = rdd_train.flatMapValues(get_convertion_timeDeltas)

In [ ]:
rdd_train_test.take(30)

In [ ]:
timedelta(days=3).total_seconds()